In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
data_path = "datasetIRB2400.csv"  # Update this path if different
data = pd.read_csv(data_path)

In [3]:
# Inspect the data
print(data.head())
print(data.info())

# Drop unnecessary columns or rows if any (customize as needed)
# For instance, if there are null values:
data = data.dropna()

         x         y          z     yaw   pitch    roll   q1_in   q2_in  \
0 -80.9148 -121.7499  1896.4867 -0.6941  0.9229 -1.4411 -2.2407 -0.7975   
1 -80.9148 -121.7499  1896.4867 -0.6941  0.9229 -1.4411 -2.2233 -0.8511   
2 -80.9148 -121.7499  1896.4867 -0.6941  0.9229 -1.4411 -2.2567 -0.9468   
3 -80.9148 -121.7499  1896.4867 -0.6941  0.9229 -1.4411 -2.1557 -0.8205   
4 -80.9148 -121.7499  1896.4867 -0.6941  0.9229 -1.4411 -2.3267 -0.9279   

    q3_in   q4_in   q5_in   q6_in  q1_out  q2_out  q3_out  q4_out  q5_out  \
0  0.6551 -3.2989  1.1212  5.7052 -2.2347 -0.8949  0.6113 -3.2841  1.2117   
1  0.5966 -3.2320  1.1662  5.6429 -2.2347 -0.8949  0.6113 -3.2841  1.2117   
2  0.5820 -3.3165  1.1220  5.5944 -2.2347 -0.8949  0.6113 -3.2841  1.2117   
3  0.5532 -3.1993  1.2697  5.7474 -2.2347 -0.8949  0.6113 -3.2841  1.2117   
4  0.6722 -3.2867  1.2266  5.6304 -2.2347 -0.8949  0.6113 -3.2841  1.2117   

   q6_out  
0  5.6792  
1  5.6792  
2  5.6792  
3  5.6792  
4  5.6792  
<class 'pandas

In [4]:
# Update the feature and target column names based on your dataset
# "q1_in","q2_in","q3_in","q4_in","q5_in", "q6_in","x", "y", "z"
X = data.drop(columns=["q1_out","q2_out","q3_out","q4_out","q5_out", "q6_out"])
y = data[["q1_out","q2_out","q3_out","q4_out","q5_out", "q6_out"]]

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save the processed data if needed
pd.DataFrame(X_train_scaled).to_csv("./X_train_scaled.csv", index=False)
pd.DataFrame(X_test_scaled).to_csv("./X_test_scaled.csv", index=False)
pd.DataFrame(y_train).to_csv("./y_train.csv", index=False)
pd.DataFrame(y_test).to_csv("./y_test.csv", index=False)

print("Preprocessing complete. Data is ready for training.")


Preprocessing complete. Data is ready for training.


In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [9]:
# # Build the neural network model
# model = Sequential([
#     Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)),
#     Dropout(0.3),
#     Dense(64, activation='relu'),
#     Dropout(0.2),
#     Dense(32, activation='relu'),
#     Dense(y_train.shape[1], activation='linear')  # Output layer for regression
# ])

# Build the model with more layers and neurons
model = Sequential([
    Dense(256, input_dim=X_train.shape[1], activation='relu'),  # Input layer
    BatchNormalization(),
    Dropout(0.3),

    Dense(512, activation='relu'),  # Hidden layer 1
    BatchNormalization(),
    Dropout(0.4),

    Dense(512, activation='relu'),  # Hidden layer 2
    BatchNormalization(),
    Dropout(0.4),

    Dense(256, activation='relu'),  # Hidden layer 3
    BatchNormalization(),
    Dropout(0.3),

    Dense(128, activation='relu'),  # Hidden layer 4
    BatchNormalization(),
    Dropout(0.3),

    Dense(64, activation='relu'),  # Hidden layer 5
    BatchNormalization(),
    Dropout(0.2),

    Dense(y_train.shape[1], activation='linear')  # Output layer
])

# Load pretrained weights if available
# model.load_weights(pretrained_weights)

# Compile the model with optimal parameters
loss = tf.keras.losses.MeanSquaredError()
optimizer = Adam(learning_rate=1e-3)  # Lower learning rate for fine-tuning
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

model.compile(optimizer=optimizer, loss=loss, metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])


c:\Users\dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

# Train the model
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=5,
    batch_size=64,
    callbacks=[early_stop],
    verbose=1
)

# Evaluate the model
loss, rmse = model.evaluate(X_test_scaled, y_test, verbose=1)
print(f"Test Loss: {loss}, Test RMSE: {rmse}")

# Save the trained model
model.save("robot_kinematics_model.h5")

Epoch 1/20
 398/3000 ━━━━━━━━━━━━━━━━━━━━ 58s 22ms/step - loss: 2.1618 - rmse: 1.4363

KeyboardInterrupt: 

In [58]:
import numpy as np

# Example single input (replace with your actual input row index or values)
single_input = np.array(X_train.iloc[0])  # Extract the row and convert to a NumPy array

# Reshape the input to ensure it's 2D (one sample with multiple features)
single_input = single_input.reshape(1, -1)  # Now it has shape (1, n_features)

# Scale the input using the same scaler as during training
single_input_scaled = scaler.transform(single_input)  # Assuming you used StandardScaler

# Predict the output using the trained model
predicted_output = model.predict(single_input_scaled)

print("Predicted output:")
print(predicted_output)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
Predicted output:
[[-0.55198735  0.8572707  -0.7072096  -2.5471854   1.2572403   1.5994673 ]]


In [70]:
import numpy as np
import pandas as pd

# Generate 20 random indices within the range of the dataset (0 to 299999)
random_indices = np.random.randint(0, 200000, size=2)

# random_indices = 23311

# Select 20 random samples from the dataset
random_samples = X_train.iloc[random_indices]  # Features
random_actual_outputs = y_train.iloc[random_indices]  # Actual target outputs

# Scale the inputs using the same scaler as during training
random_samples_scaled = scaler.transform(random_samples)

# Predict the outputs using the trained model
predicted_outputs = model.predict(random_samples_scaled)

print("Random 20 Samples:")
print(random_samples)
print("\nActual Outputs:")
print(random_actual_outputs)
print("\nPredicted Outputs:")
print(predicted_outputs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 612ms/step
Random 20 Samples:
               x         y          z     yaw   pitch    roll   q1_in   q2_in  \
185155  329.1358  920.5482   968.7923  2.2671  0.6526  0.9870 -1.9671 -1.9764   
120519   45.8579  550.7029  1159.5728 -2.4838  0.5058  1.4941  1.3230  0.3325   

         q3_in   q4_in   q5_in   q6_in  
185155 -0.1715  3.2783 -0.8620  0.6936  
120519 -0.5392 -2.1532 -1.6323  4.3153  

Actual Outputs:
        q1_out  q2_out  q3_out  q4_out  q5_out  q6_out
185155 -1.9015 -1.9070 -0.1442  3.3215 -0.9467  0.6517
120519  1.3674  0.2786 -0.5221 -2.2414 -1.6557  4.3958

Predicted Outputs:
[[-1.9598285  -1.6671988  -0.05657279  3.098069   -0.86545104  0.6834614 ]
 [ 1.4457448   0.28961384 -0.40137124 -2.2202506  -1.6317266   4.1602216 ]]


In [72]:
import numpy as np
import pandas as pd

# Generate 20 random indices within the range of the dataset (0 to 299999)
random_indices = np.random.randint(0, 200000, size=2)

# Select 20 random samples from the dataset
random_samples = X_train.iloc[random_indices]  # Features
random_actual_outputs = y_train.iloc[random_indices]  # Actual target outputs

# Scale the inputs using the same scaler as during training
random_samples_scaled = scaler.transform(random_samples)

# Predict the outputs using the trained model
predicted_outputs = model.predict(random_samples_scaled)

print("Random 20 Samples:")
print(random_samples)
print("\nActual Outputs:")
print(random_actual_outputs)
print("\nPredicted Outputs:")
print(predicted_outputs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Random 20 Samples:
               x         y         z     yaw   pitch    roll   q1_in   q2_in  \
264475  844.3646 -222.5794  815.2586 -1.2701 -0.5224 -1.3601 -0.2716  0.8224   
38103   260.1647  395.6295  995.7755  1.9977 -0.5596  1.0248 -1.9833 -1.2482   

         q3_in   q4_in   q5_in   q6_in  
264475 -0.4097 -2.2200 -1.0092  0.7062  
38103  -0.7844  0.8483  1.3975  4.4272  

Actual Outputs:
        q1_out  q2_out  q3_out  q4_out  q5_out  q6_out
264475 -0.3215   0.792 -0.3633 -2.3059 -1.0817  0.7197
38103  -2.0155  -1.195 -0.8025  0.9049  1.3162  4.4222

Predicted Outputs:
[[-0.17256698  0.76281744 -0.28731963 -2.1511936  -0.9768766   0.8064764 ]
 [-1.9987706  -1.2498145  -0.760355    0.8393113   1.407675    4.4579954 ]]
